In [1]:
import mrcfile
import numpy as np
from subprocess import call
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import os
import fnmatch
from ClassFiles.ut import create_single_folder
from ClassFiles.Framework import AdversarialRegulariser 

In [2]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name).replace("\\", "/"))
    return result

In [50]:
path = '/local/scratch/public/sl767/MRC_Data/org/'
tmp_path = '/local/scratch/public/sl767/MRC_Data/tmp/low_pass_filter/'
create_single_folder(tmp_path)
training_path = path+'training/'
eval_path = path+'eval/'

In [52]:
train_list = find('*.mrc', training_path)
train_amount = len(train_list)
print('Training Pictures found: ' + str(train_amount))
eval_list = find('*.mrc', eval_path)
eval_amount = len(eval_list)
print('Evaluation Pictures found: ' + str(eval_amount))

Training Pictures found: 2991
Evaluation Pictures found: 25


In [26]:
def get_pdb_id(path):
    return path[-8:len(path)]
def get_tmp_path(path):
    return tmp_path+get_pdb_id(path)

In [31]:
def low_pass_filter(i,o):
    call('relion_image_handler --i {} --o {} --lowpass 30 --angpix 3'.format(i, o), shell=True)

In [30]:
def adversarial(molecule_path):
    out = get_tmp_path(molecule_path)
    if os.path.isfile(out):
        return out
    else:
        low_pass_filter(molecule_path,out)
        return out

In [32]:
def get_image(number, training):
    if training:
        L = train_list
    else:
        L = eval_list
    molecule_path=L[number]
    with mrcfile.open(molecule_path) as mrc:
        gt = mrc.data
    with mrcfile.open(adversarial(molecule_path)) as mrc:
        adv = mrc.data
    return gt, adv

def load_data(training_data=True):
    if training_data:
        n = train_amount
    else:
        n = eval_amount
    return get_image(random.randint(0, n-1), training=training_data)

In [33]:
BATCH_SIZE=2
def get_batch(training_data=True):
    true = np.zeros(shape=(BATCH_SIZE, 96,96,96))
    adv = np.zeros(shape=(BATCH_SIZE, 96,96,96))
    for k in range(BATCH_SIZE):
        gt, adver = load_data(training_data=training_data)
        true[k,...]=gt
        adv[k,...]=adver
    return gt,adv

In [36]:
saves_path = '/local/scratch/public/sl767/SPA/Saves/Adversarial_Regulariser/LowPassFiltering'
regularizer = AdversarialRegulariser(saves_path)

No save found


In [38]:
def evaluate():
    gt, adv = get_batch(training_data=False)
    regularizer.test(groundTruth=gt, adversarial=adv, fourier_data =False)

LEARNING_RATE=0.001
def train(steps):
    for k in range(steps):
        gt, adv = get_batch()
        regularizer.train(groundTruth=gt, adversarial=adv, learning_rate=LEARNING_RATE, fourier_data =False)
        if k%100==0:
            evaluate()
    regularizer.save()

In [39]:
# for k in range(5):
#     train(5000)

In [40]:
# visual check for low pass filtering